In [1]:
import import_ipynb
from Helpers import *
import pandas as pd
import numpy as np
import sklearn
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import make_pipeline, Pipeline

from sklearn.model_selection import cross_val_score, GridSearchCV,cross_val_predict
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler

importing Jupyter notebook from Helpers.ipynb


In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

test_ids = test['Id']

In [3]:



# class HomePreprocessor:
#     def __init__(self, drop_low_correlated=False):
#         self.drop_low_correlated = drop_low_correlated

#     def fit(self, X, y=None):
#         return self

#     def transform(self, X, *_):

#         if (drop_low_correlated):
#             X=X.pipe(drop_non_correlated_columns, min_corr=ignoreUncorrelatedRate)
        
#         df = df.pipe(drop_columns, columns=high_nan_columns) # high number of NA values 
#         .pipe(drop_columns, columns=no_variety_columns) # no variety

#         .pipe(fill_NA, columns=columnsToFillNone, value='None') # 
#         .pipe(fill_NA, columns=columnsToFillZero, value=0) # 
#         .pipe(get_dummies_for_category_columns, columns=X.columns[X.dtypes=="object"])
#         .pipe(get_dummies_for_category_columns, columns=categoryColumns2)
#         .pipe(set_type, 'MasVnrArea','int64')
#         .pipe(year_to_age, 'YearBuilt')
#         .pipe(year_to_age, 'YearRemodAdd')
#         .pipe(year_to_age, 'YrSold')
#         .pipe(drop_columns, columns=["Id","BedroomAbvGr"])## drop Garbage
#         .pipe(log_tranform, columns=areaColumns)       
#         return X

# train = preprocess(train).pipe(drop_non_correlated_columns, min_corr=ignoreUncorrelatedRate) 
# test_ids=test['Id']
# test = preprocess(test)


In [4]:

y_train = np.log(train['SalePrice'])
#y_test = np.log(test['SalePrice'])

cs = ColumnSelector(columns=['GrLivArea', 'YearBuilt', 'OverallQual','MSSubClass'])
dt = DummiesTransformer(column='MSSubClass', keys=[ 20,  30,  40,  45,  50,  60,  70,  75,  80,  85,  90, 120, 150, 160, 180, 190])
pp = Pipeline(
    [('SelectColumns', cs),('Dummies', dt)])
X_train = pp.fit_transform(train)
X_test = pp.fit_transform(test)



# Perform Grid-Search
gsc = GridSearchCV(
    estimator=RandomForestRegressor(),
    param_grid={
        'max_depth': range(8,10),
        'n_estimators': ( 50, 100, 1000, 2000),
    },
    cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)

grid_result = gsc.fit(X_train, y_train)
best_params = grid_result.best_params_
print(best_params)
rfr = RandomForestRegressor(max_depth=best_params["max_depth"], n_estimators=best_params["n_estimators"],
                            random_state=False, verbose=False)


scores = cross_val_score(rfr, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
print(scores)

/Users/drmegavolt/miniconda3/envs/home_iowa/lib/python3.7/site-packages/sklearn/base.py:553: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  return self.fit(X, **fit_params).transform(X)
Helpers.ipynb:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  "\n",


{'max_depth': 8, 'n_estimators': 1000}
[-0.03060126 -0.02572984 -0.03170459 -0.03506124 -0.03620101 -0.02550913
 -0.02359448 -0.02473443 -0.02734947 -0.02980915]


In [5]:
#sklearn.metrics.SCORERS.keys()
rfr.fit(X_train, y_train)
predictions = rfr.predict( X_test)

In [6]:
result=np.exp(predictions)

In [7]:
result

array([124001.31958912, 152603.91794636, 166300.38093676, ...,
       141812.05039926, 129845.6675408 , 227536.10000014])

In [8]:
dfR = pd.DataFrame(test_ids,columns=['Id'])
dfR['SalePrice']=result
dfR.to_csv('predictions.csv',index = None, header=True)
print(dfR.head())

     Id      SalePrice
0  1461  124001.319589
1  1462  152603.917946
2  1463  166300.380937
3  1464  181007.865275
4  1465  202710.594751


In [13]:
test.columns[test.isnull().sum()>0]

Index(['MSZoning', 'LotFrontage', 'Alley', 'Utilities', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath',
       'BsmtHalfBath', 'KitchenQual', 'Functional', 'FireplaceQu',
       'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea',
       'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType'],
      dtype='object')

In [10]:
test['MiscFeature'].unique()

array([nan, 'Gar2', 'Shed', 'Othr'], dtype=object)

In [17]:
(test['BsmtFullBath']+0.5*test.BsmtHalfBath).isnull()

0       False
1       False
2       False
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15      False
16      False
17      False
18      False
19      False
20      False
21      False
22      False
23      False
24      False
25      False
26      False
27      False
28      False
29      False
        ...  
1429    False
1430    False
1431    False
1432    False
1433    False
1434    False
1435    False
1436    False
1437    False
1438    False
1439    False
1440    False
1441    False
1442    False
1443    False
1444    False
1445    False
1446    False
1447    False
1448    False
1449    False
1450    False
1451    False
1452    False
1453    False
1454    False
1455    False
1456    False
1457    False
1458    False
Length: 1459, dtype: bool